In [133]:
import pandas as pd
import numpy as np
import ast


In [166]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [ ]:
movies.head()

In [ ]:
movies.columns

In [ ]:
movies.dtypes

In [ ]:
credits.head()

In [ ]:
movies.shape

In [167]:
movies = movies.merge(credits, on="title")

In [ ]:
movies.shape

In [ ]:
movies.columns

In [168]:
movies = movies[[ 'genres', 'id', 'keywords',
 'overview', 'title', 'cast','crew']]

In [ ]:
movies.info()

In [ ]:
movies.head()

In [ ]:
movies.isnull().sum()

In [137]:
movies = movies.dropna()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.columns

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [169]:
def converTolist(obj):
    l = []
    for item in ast.literal_eval(obj):
        l.append(item['name'])
    return l

In [170]:
movies['genres'] = movies['genres'].apply(converTolist)

In [ ]:
movies.head()

In [171]:
movies['keywords'] = movies['keywords'].apply(converTolist)

In [172]:
def convertCast(obj):
    l = []
    d = ast.literal_eval(obj)
    for i in range(0,min(3,len(d))):
        l.append(d[i]["name"])
    return l

In [173]:
movies['cast'] = movies['cast'].apply(convertCast)

In [ ]:
movies.head()

In [ ]:
movies['crew'][0]

In [174]:
def fetch_director(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i["job"] == "Director":
            l.append(i["name"])
            break
    return l

In [175]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies["crew"]

In [ ]:
movies.info()

In [ ]:
movies.isnull().sum()

In [145]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies['crew'][0]

In [146]:
movies['overview'] = movies['overview'].apply(lambda x:[i.replace(" ","") for i in x])

In [147]:
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [148]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head(1)

In [149]:
movies["tags"] = movies["overview"]+movies["keywords"]+movies["genres"]+movies["crew"]+movies["cast"]

In [ ]:
movies.head()

In [150]:
new_df = movies[["id","title","tags"]]

In [ ]:
new_df

In [151]:
new_df["tags"] = new_df["tags"].apply(lambda x:" ".join(x))

C:\Users\meetj\AppData\Local\Temp\ipykernel_24956\1578042357.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"] = new_df["tags"].apply(lambda x:" ".join(x))


In [ ]:
new_df


In [152]:
new_df["tags"] = new_df["tags"].apply(lambda x:x.lower())

C:\Users\meetj\AppData\Local\Temp\ipykernel_24956\3481706753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"] = new_df["tags"].apply(lambda x:x.lower())


In [153]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000,stop_words="english")

In [154]:
vectors = cv.fit_transform(new_df["tags"]).toarray()

In [ ]:
vectors[0]

In [ ]:
cv.get_feature_names_out()

In [155]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [156]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)


In [157]:
new_df["tags"] = new_df["tags"].apply(stem)

C:\Users\meetj\AppData\Local\Temp\ipykernel_24956\825579503.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"] = new_df["tags"].apply(stem)


In [158]:
from sklearn.metrics.pairwise import cosine_similarity


In [159]:
similarity = cosine_similarity(vectors)

In [160]:
def recommend(movie):
    movie_index = new_df[new_df["title"] == movie].index[0]
    distances = similarity[movie_index]
    mostSimilar = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    movienames = []
    for i in mostSimilar:
        movienames.append(new_df.iloc[i[0]]["title"])
    return movienames

In [ ]:
print(recommend("Batman Begins"))

['The Dark Knight', 'The Dark Knight Rises', 'Batman', 'Batman', 'Batman & Robin']


In [162]:
import pickle

In [163]:
pickle.dump(similarity, open("similarity.pkl","wb"))

In [177]:
pickle.dump(movies, open("movies.pkl","wb"))

In [176]:
movies

,genres,id,keywords,overview,title,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]
...,...,...,...,...,...,...,...
4804,"[Action, Crime, Thriller]",9367,"[united states–mexico barrier, legs, arms, pap...",El Mariachi just wants to play his guitar and ...,El Mariachi,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...",[Robert Rodriguez]
4805,"[Comedy, Romance]",72766,[],A newlywed couple's honeymoon is upended by th...,Newlyweds,"[Edward Burns, Kerry Bishé, Marsha Dietlein]",[Edward Burns]
4806,"[Comedy, Drama, Romance, TV Movie]",231617,"[date, love at first sight, narration, investi...","""Signed, Sealed, Delivered"" introduces a dedic...","Signed, Sealed, Delivered","[Eric Mabius, Kristin Booth, Crystal Lowe]",[Scott Smith]
4807,[],126186,[],When ambitious New York attorney Sam is sent t...,Shanghai Calling,"[Daniel Henney, Eliza Coupe, Bill Paxton]",[Daniel Hsia]


In [180]:
movie_titles = new_df["title"]

In [181]:
movie_titles.to_csv("titles.csv")